# Project 4: Predicting Outbreaks of West Nile Virus in Chicago

Group 2 Joint Submission: Derrick Hoe, Joyce Ooi, Nee Bimin, Cheong Yu Chye

## Notebook 2: Pre-processing

This notebook contains the code for pre-processing, which includes feature engineering as needed.

### Contents

- [Importing of Libraries](#Importing-of-Libraries)
- [Data Import](#Data-Import)
- [Combining Training and Testing Datasets](#Combining-Training-and-Testing-Datasets)
    * [Merge Weather Dataset](#Merge-Weather-Dataset)
    * [Feature Selection](#Feature-Selection)
    * [One-Hot Encode Categorical Variable](#One-Hot-Encode-Categorical-Variable)
- [Splitting of Combined Dataset back to Training and Testing Datasets](#Splitting-of-Combined-Dataset-back-to-Training-and-Testing-Datasets)
- [Data Export](#Data-Export)

## Importing of Libraries

In [1]:
# Read in libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import scipy.stats as stats
from mpl_toolkits import mplot3d
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import ensemble, preprocessing
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline 

C:\Users\yuchy\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Data Import

In [2]:
# Import data from downloaded csv files
train = pd.read_csv('../assets/train_clean.csv')
test = pd.read_csv('../assets/test_clean.csv')
spray = pd.read_csv('../assets/spray_clean.csv')
weather = pd.read_csv('../assets/weather_clean.csv')

## Feature Engineering

In [3]:
train.head()

,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent,month,day,year,coord
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,5,29,2007,"(41.95469, -87.800991)"
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,5,29,2007,"(41.95469, -87.800991)"
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,5,29,2007,"(41.994991, -87.769279)"
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,5,29,2007,"(41.974089, -87.824812)"
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,5,29,2007,"(41.974089, -87.824812)"


In [4]:
test.head()

,id,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [5]:
spray.head()

,date,latitude,longitude
0,2011-08-29,42.391623,-88.089163
1,2011-08-29,42.391348,-88.089163
2,2011-08-29,42.391022,-88.089157
3,2011-08-29,42.390637,-88.089158
4,2011-08-29,42.390410,-88.088858


In [6]:
weather.head()

,date,tavg,dewpoint,preciptotal,wetbulb
0,2007-05-01,67.0,51,0.0,56.0
1,2007-05-01,68.0,51,0.0,57.0
2,2007-05-02,51.0,42,0.0,47.0
3,2007-05-02,52.0,42,0.0,47.0
4,2007-05-03,56.0,40,0.0,48.0


In [7]:
weather.shape

(2899, 5)

In [8]:
weather['date'].nunique()

1472

## Combining the Training and Testing Datasets

In [9]:
# Add temporary column to both train and test to facilitate with splitting later
train['dataset'] = 'train'
test['dataset'] = 'test'

In [10]:
common_cols = ['id',\
               'wnvpresent',\
               'street',\
               'date',\
               'latitude',\
               'addressnumberandstreet',\
               'species',\
               'address',\
               'addressaccuracy',\
               'longitude',\
               'trap',\
               'block',\
               'dataset']

In [11]:
train.shape, test.shape

((9693, 17), (116293, 12))

In [12]:
combined = pd.concat([train,test], ignore_index=True)

In [13]:
combined.head()

,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent,month,day,year,coord,dataset,id
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1.0,0.0,5.0,29.0,2007.0,"(41.95469, -87.800991)",train,NaN
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1.0,0.0,5.0,29.0,2007.0,"(41.95469, -87.800991)",train,NaN
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1.0,0.0,5.0,29.0,2007.0,"(41.994991, -87.769279)",train,NaN
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1.0,0.0,5.0,29.0,2007.0,"(41.974089, -87.824812)",train,NaN
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4.0,0.0,5.0,29.0,2007.0,"(41.974089, -87.824812)",train,NaN


In [14]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125986 entries, 0 to 125985
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   date                    125986 non-null  object 
 1   address                 125986 non-null  object 
 2   species                 125986 non-null  object 
 3   block                   125986 non-null  int64  
 4   street                  125986 non-null  object 
 5   trap                    125986 non-null  object 
 6   addressnumberandstreet  125986 non-null  object 
 7   latitude                125986 non-null  float64
 8   longitude               125986 non-null  float64
 9   addressaccuracy         125986 non-null  int64  
 10  nummosquitos            9693 non-null    float64
 11  wnvpresent              9693 non-null    float64
 12  month                   9693 non-null    float64
 13  day                     9693 non-null    float64
 14  year                

### Feature Selection

Features such as address, block, street, addressnumberandstreet and addressaccuracy will be dropped because they can be represented by latitiude and longitude.

In [15]:
combined = combined[['id', 'date', 'species', 'trap', 'latitude', 'longitude', 'nummosquitos',
                     'dataset', 'wnvpresent']]
combined.head()

,id,date,species,trap,latitude,longitude,nummosquitos,dataset,wnvpresent
0,NaN,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,1.0,train,0.0
1,NaN,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,1.0,train,0.0
2,NaN,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,1.0,train,0.0
3,NaN,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,1.0,train,0.0
4,NaN,2007-05-29,CULEX RESTUANS,T015,41.974089,-87.824812,4.0,train,0.0


In [16]:
combined.shape

(125986, 9)

### Merge Weather Dataset

The weather dataset is next merged with the combined dataset based on the dates so that we can include it in our analysis. 

In [17]:
weather.head()

,date,tavg,dewpoint,preciptotal,wetbulb
0,2007-05-01,67.0,51,0.0,56.0
1,2007-05-01,68.0,51,0.0,57.0
2,2007-05-02,51.0,42,0.0,47.0
3,2007-05-02,52.0,42,0.0,47.0
4,2007-05-03,56.0,40,0.0,48.0


In [18]:
# Find the number of unique values of date
weather['date'].nunique()

1472

In [19]:
# 'Group' the rows having similar dates while assigning mean values for the numerical columns, for each group
weather = weather.groupby('date').mean().reset_index()

In [20]:
weather.shape

(1472, 5)

In [21]:
weather.head()

,date,tavg,dewpoint,preciptotal,wetbulb
0,2007-05-01,67.50,51.0,0.0,56.5
1,2007-05-02,51.50,42.0,0.0,47.0
2,2007-05-03,57.00,40.0,0.0,49.0
3,2007-05-04,61.25,41.5,0.0,50.0
4,2007-05-05,60.00,38.5,0.0,49.5


In [22]:
combined = pd.merge(combined, weather, how ='inner', on ='date')

Based on the correlation analysis done during EDA, we noticed that the following pairs of features were highly correlated:
- wetbulb and tavg
- wetbub and dewpoint

In view of this, we add two new features:
- wetbulb * tavg
- wetbulb * dewpoint

We also add a third feature:
- tavg * wetbulb * dewpoint

We remove the original features: tavg, wetbulb and dewpoint.

In [23]:
combined.head()

,id,date,species,trap,latitude,longitude,nummosquitos,dataset,wnvpresent,tavg,dewpoint,preciptotal,wetbulb
0,NaN,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,1.0,train,0.0,75.5,58.5,0.0,65.5
1,NaN,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,1.0,train,0.0,75.5,58.5,0.0,65.5
2,NaN,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,1.0,train,0.0,75.5,58.5,0.0,65.5
3,NaN,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,1.0,train,0.0,75.5,58.5,0.0,65.5
4,NaN,2007-05-29,CULEX RESTUANS,T015,41.974089,-87.824812,4.0,train,0.0,75.5,58.5,0.0,65.5


In [24]:
# Create new features
combined['tavg*wetbulb'] = combined['tavg'] * combined['wetbulb']
combined['wetbulb*dewpoint'] = combined['wetbulb'] * combined['dewpoint']
combined['tavg*wetbulb*dewpoint'] = combined['tavg'] * combined['wetbulb'] * combined['dewpoint']

In [25]:
combined.drop(columns=['tavg', 'wetbulb', 'dewpoint', 'trap'], inplace=True) # drop the original columns

### One-Hot Encode Categorical Variable

Since species of mosquitoes can be divided into 7 distinct species, we will convert it into dummy variables to be able to continue with modelling.

In [26]:
# Create dummy columns for species
combined = pd.get_dummies(combined, columns=['species'], drop_first=True)

In [27]:
combined.head()

,id,date,latitude,longitude,nummosquitos,dataset,wnvpresent,preciptotal,tavg*wetbulb,wetbulb*dewpoint,tavg*wetbulb*dewpoint,species_CULEX PIPIENS,species_CULEX PIPIENS/RESTUANS,species_CULEX RESTUANS,species_CULEX SALINARIUS,species_CULEX TARSALIS,species_CULEX TERRITANS,species_UNSPECIFIED CULEX
0,NaN,2007-05-29,41.954690,-87.800991,1.0,train,0.0,0.0,4945.25,3831.75,289297.125,0,1,0,0,0,0,0
1,NaN,2007-05-29,41.954690,-87.800991,1.0,train,0.0,0.0,4945.25,3831.75,289297.125,0,0,1,0,0,0,0
2,NaN,2007-05-29,41.994991,-87.769279,1.0,train,0.0,0.0,4945.25,3831.75,289297.125,0,0,1,0,0,0,0
3,NaN,2007-05-29,41.974089,-87.824812,1.0,train,0.0,0.0,4945.25,3831.75,289297.125,0,1,0,0,0,0,0
4,NaN,2007-05-29,41.974089,-87.824812,4.0,train,0.0,0.0,4945.25,3831.75,289297.125,0,0,1,0,0,0,0


## Splitting of Combined Dataset back to Training and Testing Datasets

After splitting, we will drop some of the columns:
* dataset - to drop from both sets because we created this earlier to identify the training and test sets
* id - to drop from train set because it was not originally there
* wnvpresent - to drop on test set because it was not originally there
* nummosquitos - to drop from both because it was only present in training set but was not present in test set; we need to ensure the variables present in both sets are the same

Note that the spray dataset is not included in our analysis because it would only have a large impact during one of the training set years. 

In [28]:
# Now split back to train and test
train_preproc = combined[combined['dataset']=='train'].copy()
train_preproc.reset_index(inplace=True, drop=True)
train_preproc.drop(['dataset', 'id', 'nummosquitos'], axis=1, inplace=True)

test_preproc = combined[combined['dataset']=='test'].copy()
test_preproc.reset_index(inplace=True, drop=True)
test_preproc.drop(['dataset', 'wnvpresent', 'nummosquitos'], axis=1, inplace=True)

In [29]:
train_preproc.head()

,date,latitude,longitude,wnvpresent,preciptotal,tavg*wetbulb,wetbulb*dewpoint,tavg*wetbulb*dewpoint,species_CULEX PIPIENS,species_CULEX PIPIENS/RESTUANS,species_CULEX RESTUANS,species_CULEX SALINARIUS,species_CULEX TARSALIS,species_CULEX TERRITANS,species_UNSPECIFIED CULEX
0,2007-05-29,41.954690,-87.800991,0.0,0.0,4945.25,3831.75,289297.125,0,1,0,0,0,0,0
1,2007-05-29,41.954690,-87.800991,0.0,0.0,4945.25,3831.75,289297.125,0,0,1,0,0,0,0
2,2007-05-29,41.994991,-87.769279,0.0,0.0,4945.25,3831.75,289297.125,0,0,1,0,0,0,0
3,2007-05-29,41.974089,-87.824812,0.0,0.0,4945.25,3831.75,289297.125,0,1,0,0,0,0,0
4,2007-05-29,41.974089,-87.824812,0.0,0.0,4945.25,3831.75,289297.125,0,0,1,0,0,0,0


In [30]:
test_preproc.head()

,id,date,latitude,longitude,preciptotal,tavg*wetbulb,wetbulb*dewpoint,tavg*wetbulb*dewpoint,species_CULEX PIPIENS,species_CULEX PIPIENS/RESTUANS,species_CULEX RESTUANS,species_CULEX SALINARIUS,species_CULEX TARSALIS,species_CULEX TERRITANS,species_UNSPECIFIED CULEX
0,1.0,2008-06-11,41.95469,-87.800991,0.0,4800.0,3552.0,266400.0,0,1,0,0,0,0,0
1,2.0,2008-06-11,41.95469,-87.800991,0.0,4800.0,3552.0,266400.0,0,0,1,0,0,0,0
2,3.0,2008-06-11,41.95469,-87.800991,0.0,4800.0,3552.0,266400.0,1,0,0,0,0,0,0
3,4.0,2008-06-11,41.95469,-87.800991,0.0,4800.0,3552.0,266400.0,0,0,0,1,0,0,0
4,5.0,2008-06-11,41.95469,-87.800991,0.0,4800.0,3552.0,266400.0,0,0,0,0,0,1,0


## Data Export

In [31]:
train_preproc.shape, test_preproc.shape

((9693, 15), (116293, 15))

In [32]:
train_preproc.to_csv("../assets/train_preproc.csv", index=False)
test_preproc.to_csv("../assets/test_preproc.csv", index=False)